In [2]:
import dash
import dash_core_components as dcc 
import dash_html_components as html 
import dash_table  
from dash.dependencies import Input, Output
import plotly.graph_objs as go 
import plotly.figure_factory as ffimport 
import pandas as pd
import plotly.graph_objs as go
import numpy as np
import itertools
import ccxt
import json
import datetime as dt 

# Define my world
spot_exchanges = ['bitfinex','bitstamp','coinbasepro','kraken','liquid','gemini','binance',
                    'bitbank','huobipro','poloniex','bithumb','bittrex','kucoin2']

exch_dict={}
for x in spot_exchanges:
    exec('exch_dict[x]=ccxt.{}()'.format(x))

Xpto= ['BTC','ETH','XRP','XMR','BCH','EOS','USDT','USDC','TRX','XLM','BSV','XBT','CSP','DAI']
Fiat=['USD','EUR','GBP','CHF','HKD','JPY','CNH','KRW']
xpto_fiat = [xpto+'/'+ fiat for xpto in Xpto for fiat in Fiat]
xpto_xpto = [p[0]+'/'+p[1] for p in itertools.permutations(Xpto,2)]

all_pairs = set(sum(itertools.chain([*exch_dict[x].load_markets()] for x in exch_dict),[])) 
pairs = list(set(xpto_fiat + xpto_xpto) & set(all_pairs))
pairs.sort()

def get_exchanges_for_pair(pair):
    '''input: a pair
    output: a dictionary of ccxt exchange objects of the exchanges listing the pair
    '''
    return {x:exch_dict[x] for x in exch_dict if pair in list(exch_dict[x].load_markets().keys())}

def get_pairs_for_exchange(ex):
    '''input: an exchange
    output: a list of pairs '''
    d={}
    exec('d[ex]=ccxt.{}()'.format(ex))
    d[ex].load_markets()
    return d[ex].symbols

def get_order_books(pair,ex):
    '''pair is the pair string ,'BTC/USD'...
        returns a dictionary of order books for the pair
        special case for binance which API fails if # of parmaeters > 2
    '''
    nobinance= {key:value for key, value in ex.items() if key != 'binance'and  key != 'bitfinex'}
    order_books = {key: value.fetch_order_book(pair,limit=2000 if key!='bithumb' else 50,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in nobinance.items() }
    if 'binance' in ex:
        order_books['binance'] =  ex['binance'].fetch_order_book(pair,limit=1000)
    if 'bitfinex' in ex:
        order_books['bitfinex'] =  ex['bitfinex'].fetch_order_book(pair,limit=2000)
    return order_books

def aggregate_order_books(dict_of_order_books):
    '''dict_of_order_books is a dict of ccxt like order_books
        retuns a ccxt like dictionary order book sorted by prices 
    '''
    agg_dict_order_book = {}
    bids = []
    for x in dict_of_order_books:
        for bid in dict_of_order_books[x]['bids']:
            bids.append(bid+[x])
    asks = []
    for x in dict_of_order_books:
        for ask in dict_of_order_books[x]['asks']:
            asks.append(ask+[x])
    agg_dict_order_book['bids'] = (pd.DataFrame(bids)).sort_values(by=0,ascending=False).values.tolist()
    agg_dict_order_book['asks'] = (pd.DataFrame(asks)).sort_values(by=0,ascending=True).values.tolist()
    return agg_dict_order_book

def normalize_order_book(order_book,cutoff=.1,step=.001):
    '''order_book is a dictionary with keys bids asks timestamp datetime ...
    where bids is a list of list [[bid,bid_size]] and 
    asks is a list of list [[ask,ask_size]]
    this is returned by ccxt.'exchange'.fetch_order_book()
    returns a dataframe with columns [ask, ask_size, ask_size_$, cum_ask_size_$, bid_, bid_size, bid_size_$, cum_bid_size_$]
    and an index of shape np.linspace(1 - cutoff,1 + cutoff ,step =.001 ~ 10 bps)  
    '''
    try:
        rounding = int(np.ceil(-np.log(step)/np.log(10)))
        agg = True
    except:
        agg = False
    bid_side = pd.DataFrame(order_book['bids'],columns=['bid','bid_size','exc'])
    bid_side['cum_bid_size'] = bid_side['bid_size'].cumsum()
    ask_side = pd.DataFrame(order_book['asks'],columns=['ask','ask_size','exc'])
    ask_side['cum_ask_size'] = ask_side['ask_size'].cumsum()
    ref = (bid_side['bid'][0]+ask_side['ask'][0])/2
    bid_side['bid%'] = round(bid_side['bid']/ref,rounding) if agg else bid_side['bid']/ref
    ask_side['ask%'] = round(ask_side['ask']/ref,rounding) if agg else ask_side['ask']/ref
    bid_side = bid_side[bid_side['bid%']>=1-cutoff]
    ask_side = ask_side[ask_side['ask%']<=1+cutoff]
    bid_side['bid_size_$'] = bid_side['bid_size']*bid_side['bid']
    bid_side['cum_bid_size_$'] = bid_side['bid_size_$'].cumsum()
    ask_side['ask_size_$'] = ask_side['ask_size']*ask_side['ask']
    ask_side['cum_ask_size_$'] = ask_side['ask_size_$'].cumsum()
    normalized_bids = pd.DataFrame(bid_side.groupby('bid%',sort=False).mean()['bid'])
    normalized_bids.columns = ['bid']
    normalized_bids['bid_size'] = bid_side.groupby('bid%',sort=False).sum()['bid_size']
    normalized_bids['cum_bid_size'] = normalized_bids['bid_size'].cumsum()
    normalized_bids['bid_size_$'] = bid_side.groupby('bid%',sort=False).sum()['bid_size_$']
    normalized_bids['cum_bid_size_$'] = normalized_bids['bid_size_$'].cumsum()
    normalized_bids['average_bid_fill'] = normalized_bids['cum_bid_size_$']/normalized_bids['cum_bid_size']
    normalized_bids['bids_exc']=bid_side.groupby('bid%',sort=False).apply(lambda x: x['exc'].loc[x['bid_size'].idxmax()])
    normalized_asks = pd.DataFrame(ask_side.groupby('ask%',sort=False).mean()['ask'])
    normalized_asks.columns = ['ask']
    normalized_asks['ask_size'] = ask_side.groupby('ask%',sort=False).sum()['ask_size']
    normalized_asks['cum_ask_size'] = normalized_asks['ask_size'].cumsum()
    normalized_asks['ask_size_$'] = ask_side.groupby('ask%',sort=False).sum()['ask_size_$']
    normalized_asks['cum_ask_size_$'] = normalized_asks['ask_size_$'].cumsum()
    normalized_asks['average_ask_fill']=normalized_asks['cum_ask_size_$']/normalized_asks['cum_ask_size']
    normalized_asks['asks_exc']=ask_side.groupby('ask%',sort=False).apply(lambda x: x['exc'].loc[x['ask_size'].idxmax()])
    book=pd.concat([normalized_asks,normalized_bids],sort=False)
    return book

def build_book(order_books,pair,exchanges,cutoff=.1,step=0.001):
    ''' gets order books aggreagtes them then normalizes
        returns a dataframe
    '''
    return normalize_order_book(aggregate_order_books({key:order_books[key] for key in exchanges}),cutoff,step)

def plot_book(order_books,pair, exc, relative=True, currency=True, cutoff=.1):
    ''' plots the order book as a v shape chart '''
    order_book = build_book(order_books,pair,exc,cutoff)
    best_bid = round(order_book['bid'].max(),4)
    best_ask = round(order_book['ask'].min(),4)
    if currency:
        col_to_chart = '_$'
    else:
        col_to_chart = ''
    if relative:
        trace_asks=go.Scatter(x=order_book.index,y=order_book['cum_ask_size'+col_to_chart],
                        name='asks',marker=dict(color='rgba(255,0,0,0.6)'),fill='tozeroy',fillcolor='rgba(255,0,0,0.2)')
        trace_bids=go.Scatter(x=order_book.index,y=order_book['cum_bid_size'+col_to_chart],
                        name='asks',marker=dict(color='rgba(0,0,255,0.6)'),fill='tozeroy',fillcolor='rgba(0,0,255,0.2)')     
    else:
        trace_asks=go.Scatter(x=order_book['ask'].fillna(0)+order_book['bid'].fillna(0),y=order_book['cum_ask_size'+col_to_chart],
                        name='asks',marker=dict(color='rgba(255,0,0,0.6)'),fill='tozeroy',fillcolor='rgba(255,0,0,0.15)')
        trace_bids=go.Scatter(x=order_book['ask'].fillna(0)+order_book['bid'].fillna(0),y=order_book['cum_bid_size'+col_to_chart],
                        name='asks',marker=dict(color='rgba(0,0,255,0.6)'),fill='tozeroy',fillcolor='rgba(0,0,255,0.15)')
        
    layout = go.Layout(title = ' - '.join(exc), xaxis = dict(title= pair +'  ' + str(best_bid)+' - '+ str(best_ask)))
    data=[trace_asks,trace_bids]
    figure = go.Figure(data=data,layout=layout)
    return figure

def plot_depth(order_books,pair, exc, relative=True, currency=True, cutoff=.1):
    if currency:
        col_to_chart = '_$'
    else:
        col_to_chart = ''
    order_book = build_book(order_books,pair,exc,cutoff)
    mid = (order_book['bid'].max()+order_book['ask'].min())/2 if relative else 1
    trace_asks = go.Scatter(x=order_book['cum_ask_size'+col_to_chart],y=order_book['average_ask_fill']/mid,
                        name='ask depth',marker=dict(color='rgba(255,0,0,0.6)'),fill='tozerox',fillcolor='rgba(255,0,0,0.15)')
    trace_bids = go.Scatter(x=-order_book['cum_bid_size'+col_to_chart],y=order_book['average_bid_fill']/mid,
                        name='bid depth',marker=dict(color='rgba(0,0,255,0.6)'),fill='tozerox',fillcolor='rgba(0,0,255,0.15)')
    data = [trace_asks,trace_bids]
    figure = go.Figure(data=data, layout={'title': 'Market Depth'})
    return figure

def order_fill(order_book_df, order_sizes,in_ccy=True):
    '''takes in an order book dataframe and an np.array of order sizes
        with size in currecncy by default else in coin
        returns an np.array of the purchase costs or the sale proceeds of an order
    '''
    average_fills = np.zeros(order_sizes.shape)
    mid=(order_book_df['ask'].min()+order_book_df['bid'].max())/2
    if in_ccy:
        order_sizes=order_sizes/mid
    for i , order_size in enumerate(order_sizes):
        if order_size > 0:
            try:
                last_line = order_book_df[order_book_df['cum_ask_size']>order_size].iloc[0]
                ccy_fill = last_line['cum_ask_size_$']+(order_size-last_line['cum_ask_size'])*last_line['ask']
                average_fill=ccy_fill/order_size
            except:
                average_fill=np.nan
        elif order_size < 0:
            try:
                last_line = order_book_df[order_book_df['cum_bid_size'] > -order_size].iloc[0]
                ccy_fill=last_line['cum_bid_size_$']+(-order_size-last_line['cum_bid_size'])*last_line['bid']
                average_fill = -ccy_fill/order_size
            except:
                average_fill = np.nan
        average_fills[i] = average_fill
    return average_fills/mid
 
def get_liq_params(normalized,pair):
    #coin stats
    coinmar = ccxt.coinmarketcap()
    coindata=coinmar.load_markets()
    total_coins=float(coindata[pair]['info']['available_supply'])  #number of coins floating
    order_span = (1,10,20,30,40)
    clip = total_coins/100000                                      #my standard order size 
    ordersizes=np.array([clip* i for i in order_span]+[-clip* i for i in order_span]).astype(int)
    slippage = ((order_fill(normalized,ordersizes,False)-1)*100).round(2)
    #order book
    best_bid = normalized['bid'].max()
    best_ask = normalized['ask'].min()
    mid= (best_bid + best_ask)/2
    spread = best_ask-best_bid
    spread_pct = spread/mid*100
    cross = min(0,spread)
    cross_pct = min(0,spread_pct)
    #arb
    arb_ask = normalized[normalized['ask'] < best_bid]
    arb_bid = normalized[normalized['bid'] > best_ask]
    print('length_bid',len(arb_bid),'len_ask',len(arb_ask))
    if len(arb_bid) == 0:
        arb_dollar = 0
    else:
        end_bid = arb_bid.iloc[-1]['cum_bid_size']
        end_ask = arb_ask.iloc[-1]['cum_ask_size']
        bid_to_empty = end_bid <= end_ask
        if bid_to_empty:
            arb_dollar = -order_fill(normalized,np.array([end_bid]),False)[0]*mid*end_bid + arb_bid.iloc[-1]['cum_bid_size_$']
        else:
            arb_dollar =-order_fill(normalized,np.array([-end_ask]),False)[0]*mid*end_ask + arb_ask.iloc[-1]['cum_ask_size_$']

    result1 = pd.DataFrame([best_bid,best_ask,mid,spread,spread_pct,cross,cross_pct,int(arb_dollar)],
    index=['bid','ask','mid','spread','spread%','cross','cross%','arb_$']).T
    result2 = pd.DataFrame(slippage,index=[str(o) for o in ordersizes]).T
    info = coindata[pair]['info']
    select_info=['symbol','rank','24h_volume_usd','market_cap_usd',
                'available_supply','percent_change_1h','percent_change_24h','percent_change_7d']
    selected_info={key:value for key,value in info.items() if key in select_info}
    result3 = pd.DataFrame(pd.Series(selected_info)).T
    result3.columns=['Coin','Rank','24H % Volume','USD Market Cap M$','Coins Supply M','% 1h','% 24h','% 7d']
    result3['24H % Volume']= round(float(result3['24H % Volume'])/float(result3['USD Market Cap M$'])*100,1)
    result3['USD Market Cap M$'] = round(float(result3['USD Market Cap M$'])/(1000*1000),0)
    result3['Coins Supply M'] = round(float(result3['Coins Supply M'])/(1000*1000),1)
    return [result1,result2,result3]

In [3]:
pair = 'BTC/USD'
exc=get_exchanges_for_pair(pair)

In [25]:
order_books=get_order_books(pair,exc)

In [5]:
order_books

{'bitstamp': {'bids': [[3839.1, 0.043363],
   [3838.15, 1.0],
   [3837.59, 1.0],
   [3837.24, 1.0],
   [3836.72, 0.2807],
   [3836.68, 0.36],
   [3836.38, 1.0],
   [3836.01, 0.03],
   [3835.19, 4.88],
   [3835.04, 0.67765036],
   [3835.0, 2.0],
   [3834.87, 3.9036],
   [3834.41, 4.0],
   [3834.07, 10.0],
   [3834.0, 2.60824204],
   [3833.84, 3.0],
   [3833.78, 3.92],
   [3833.35, 0.42],
   [3833.34, 2.0],
   [3833.0, 3.5],
   [3832.85, 0.36],
   [3832.51, 4.1989091],
   [3831.99, 4.00019336],
   [3831.98, 5.0],
   [3831.5, 2.949615],
   [3831.0, 3.0],
   [3830.93, 2.523],
   [3830.9, 2.82447],
   [3830.42, 0.8],
   [3830.33, 4.50181324],
   [3830.0, 0.00206791],
   [3829.98, 0.4],
   [3829.8, 3.0],
   [3829.2, 0.70043691],
   [3829.03, 4.97275269],
   [3829.01, 0.36],
   [3828.32, 0.30647026],
   [3828.19, 4.6731],
   [3828.0, 0.44040998],
   [3827.91, 0.02925326],
   [3827.88, 0.090526],
   [3827.09, 0.36],
   [3826.85, 0.91],
   [3826.84, 4.5025],
   [3826.6, 2.502],
   [3826.49, 2.3

In [6]:
aggregated = aggregate_order_books(order_books)

In [7]:
normalized=normalize_order_book(aggregated)

In [8]:
normalized

,ask,ask_size,cum_ask_size,ask_size_$,cum_ask_size_$,average_ask_fill,asks_exc,bid,bid_size,cum_bid_size,bid_size_$,cum_bid_size_$,average_bid_fill,bids_exc
0.987,3839.324000,40.195358,40.195358,1.543306e+05,1.543306e+05,3839.512771,kraken,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.988,3842.708043,83.816506,124.011864,3.220708e+05,4.764013e+05,3841.578790,bitstamp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.989,3846.343495,323.747862,447.759726,1.245309e+06,1.721710e+06,3845.165079,coinbasepro,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.990,3850.661734,197.423813,645.183539,7.600967e+05,2.481807e+06,3846.667865,bitstamp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.991,3854.283071,105.473699,750.657238,4.065615e+05,2.888368e+06,3847.785844,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.992,3858.226067,643.718105,1394.375343,2.482828e+06,5.371196e+06,3852.044799,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.993,3861.693668,196.229797,1590.605140,7.579760e+05,6.129172e+06,3853.358762,kraken,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.994,3865.917444,108.651735,1699.256875,4.199312e+05,6.549103e+06,3854.098574,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.995,3869.440318,193.454861,1892.711736,7.487010e+05,7.297804e+06,3855.740062,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.996,3873.725137,32.974558,1925.686294,1.277403e+05,7.425545e+06,3856.051101,bittrex,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
order_books.keys()

dict_keys(['bitstamp', 'coinbasepro', 'kraken', 'liquid', 'gemini', 'bittrex', 'bitfinex'])

In [10]:
from plotly.offline import plot, iplot , init_notebook_mode
init_notebook_mode(connected=True)

In [11]:
dplot=plot_book(order_books,pair,exc)

In [12]:
dplot2=plot_depth(order_books,pair,exc)

In [13]:
#iplot(dplot)

In [14]:
#iplot(dplot2)

In [15]:
get_liq_params(normalized,pair)[0]

length_bid 26 len_ask 26


,bid,ask,mid,spread,spread%,cross,cross%,arb_$
0,3939.353846,3839.324,3889.338923,-100.029846,-2.571898,-100.029846,-2.571898,63079.0


In [16]:
get_liq_params(normalized,pair)[1]

length_bid 26 len_ask 26


,175,1757,3515,5272,7030,-175,-1757,-3515,-5272,-7030
0,-1.18,-0.89,-0.32,0.4,1.1,1.25,0.18,-0.67,-1.3,-2.06


In [17]:
get_liq_params(normalized,pair)[2]

length_bid 26 len_ask 26


,Coin,Rank,24H % Volume,USD Market Cap M$,Coins Supply M,% 1h,% 24h,% 7d
0,BTC,1,13.5,68568.0,17.6,-0.08,0.14,0.82


In [18]:
#order book
best_bid = normalized['bid'].max()
best_ask = normalized['ask'].min()
mid= (best_bid + best_ask)/2
spread = best_ask-best_bid
spread_pct = spread/mid*100
cross = min(0,spread)
cross_pct = min(0,spread_pct)
#arb
arb_ask = normalized[normalized['ask'] < best_bid]
arb_bid = normalized[normalized['bid'] > best_ask]
if len(arb_bid) == 0:
    print('no arb')
    arb_dollar = 0
    arb_size = 0
else:
    end_bid = arb_bid.iloc[-1]['cum_bid_size']
    end_ask = arb_ask.iloc[-1]['cum_ask_size']
    bid_to_empty = end_bid <= end_ask
    if bid_to_empty:
        print('selling first')
        arb_size = arb_bid.iloc[-1]['cum_bid_size']
        arb_dollar = -order_fill(normalized,np.array([end_bid]),False)[0]*mid*end_bid + arb_bid.iloc[-1]['cum_bid_size_$']
    else:
        print('buying first')
        arb_size = arb_ask.iloc[-1]['cum_ask_size']
        arb_dollar =-order_fill(normalized,np.array([-end_ask]),False)[0]*mid*end_ask + arb_ask.iloc[-1]['cum_ask_size_$']

result1 = pd.DataFrame([best_bid,best_ask,mid,spread,spread_pct,cross,cross_pct,arb_dollar,arb_size],
index=['bid','ask','mid','spread','spread%','cross','cross%','arb_$','arb_size']).T

selling first


In [19]:
best_bid

3939.353846153846

In [20]:
best_ask

3839.324

In [21]:
arb_bid

,ask,ask_size,cum_ask_size,ask_size_$,cum_ask_size_$,average_ask_fill,asks_exc,bid,bid_size,cum_bid_size,bid_size_$,cum_bid_size_$,average_bid_fill,bids_exc
1.013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3939.353846,95.047028,95.047028,3.744875e+05,3.744875e+05,3940.023127,bitfinex
1.012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3935.812500,63.091270,158.138298,2.482916e+05,6.227791e+05,3938.192649,bitfinex
1.011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3932.128125,95.248277,253.386575,3.745768e+05,9.973559e+05,3936.104028,bitfinex
1.010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3928.220000,84.003653,337.390229,3.299874e+05,1.327343e+06,3934.148527,bitfinex
1.009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3924.336000,49.840358,387.230586,1.955965e+05,1.522940e+06,3932.901514,bitfinex
1.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3920.474074,71.291299,458.521885,2.794903e+05,1.802430e+06,3930.957441,bitfinex
1.007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3916.859259,140.701930,599.223815,5.511703e+05,2.353600e+06,3927.748361,bitfinex
1.006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3912.667857,70.869990,670.093805,2.773028e+05,2.630903e+06,3926.171483,bitfinex
1.005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3908.802632,54.619110,724.712914,2.134966e+05,2.844400e+06,3924.864222,bitfinex
1.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3904.918919,43.011395,767.724309,1.679612e+05,3.012361e+06,3923.753518,bitfinex


In [22]:
arb_ask

,ask,ask_size,cum_ask_size,ask_size_$,cum_ask_size_$,average_ask_fill,asks_exc,bid,bid_size,cum_bid_size,bid_size_$,cum_bid_size_$,average_bid_fill,bids_exc
0.987,3839.324000,40.195358,40.195358,1.543306e+05,1.543306e+05,3839.512771,kraken,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.988,3842.708043,83.816506,124.011864,3.220708e+05,4.764013e+05,3841.578790,bitstamp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.989,3846.343495,323.747862,447.759726,1.245309e+06,1.721710e+06,3845.165079,coinbasepro,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.990,3850.661734,197.423813,645.183539,7.600967e+05,2.481807e+06,3846.667865,bitstamp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.991,3854.283071,105.473699,750.657238,4.065615e+05,2.888368e+06,3847.785844,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.992,3858.226067,643.718105,1394.375343,2.482828e+06,5.371196e+06,3852.044799,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.993,3861.693668,196.229797,1590.605140,7.579760e+05,6.129172e+06,3853.358762,kraken,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.994,3865.917444,108.651735,1699.256875,4.199312e+05,6.549103e+06,3854.098574,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.995,3869.440318,193.454861,1892.711736,7.487010e+05,7.297804e+06,3855.740062,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.996,3873.725137,32.974558,1925.686294,1.277403e+05,7.425545e+06,3856.051101,bittrex,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
arb_size

2066.61795825